# Switches Challenge

You wind up in a very long, narrow, and circular hallway. The hallway has switches on the wall, but no other features
that allow you to identify where you are in the hallway. These switches are all at the same height, and all have the
same distance of 1 step to their left and right neighbor switch. You can see if a switch is up or down, and toggle it,
but you are not allowed to mark them or their location (e.g. you cannot drop a shoe in the hallway or rip a switch out
of the wall). You are alone in the hallway. If you toggle a switch, it will stay fixed unless you toggle it again.

Now here is the problem: you can only leave the hallway after you figured out how many switches are in the hallway. How
do you do that?

# Data Structures

First, let's define a data structure for switches. Every switch only has one property, and we initialize that using
`Random.nextBoolean` to simplify the usage.

In [10]:
enum class Position { UP, DOWN }
class Switch {
    var position = if (Random.nextBoolean()) Position.UP else Position.DOWN
}

Next we create the hallway. The proper data structure to encode the problem is to use a double-linked list. We
also add a `addRight` method which adds a new node to the right side of a node.

In [11]:
class Node<T>(val data: T) {
    var left = this
    var right = this

    fun addRight(other: Node<T>) {
        other.right = right

        other.left = this
        other.right.left = other
        right = other
    }
}

Next step is that we have to create list using these data structures.  We allow the caller to pick the length of the
list, but also provide a reasonable default value.  After creating the list, we walk to the right a random number of
steps to implement the "You ended up somewhere in the hallway" situation. 

In [12]:
fun setup(count: Int): Node<Switch> {
    val first = Node(Switch())
    repeat(count - 1) { first.addRight(Node(Switch())) }
    var start = first
    repeat(Random.nextInt(count)) { start = start.right }
    return start
}

Now comes the interesting part: how do we implement the `counter` method? Here is the
basic idea: we make sure that all but one switch are in the down position. Then we walk
from that "UP" switch until we reach it again, and count how many switches we pass during
that walk, which will give us the requested answer.  Which now brings up the next issue: how
can we be sure that - when we find a switch in the UP position - it was the starting switch
and not a switch we never walked by before? The solution for this is: walk from the start into
the same direction until we find a "UP switch.  Once we arrive at a "UP" switch, we toggle it
into "DOWN" and then walk back the same number of steps.  If the switch we started at is now
"DOWN", then we toggled it before we walked back, and thus we have the answer! Otherwise,
we know that the switch we toggled was not the start switch. So we start again...

One complete initial solution is:

In [14]:
class Counter {
    enum class Position { UP, DOWN }
    class Switch {
        var position = if (Random.nextBoolean()) Position.UP else Position.DOWN
    }

    class Node<T>(val data: T) {
        var left = this
        var right = this

        fun addRight(other: Node<T>) {
            other.right = right

            other.left = this
            other.right.left = other
            right = other
        }
    }

    fun setup(count: Int): Node<Switch> {
        val first = Node(Switch())
        repeat(count - 1) { first.addRight(Node(Switch())) }
        var start = first
        repeat(Random.nextInt(count)) { start = start.right }
        return start
    }

    fun basic(count: Int = 100) {
        val start = setup(count)
        val answer = basicCount(start)
        println("The number of switches is $answer")
    }
    
    fun basicCount(start: Node<Switch>): Int {
        start.data.position = Position.UP
        var node = start
        while (true) {
            var steps = 0

            do {
                node = node.right
                steps++
            } while (node.data.position == Position.DOWN)
            node.data.position = Position.DOWN

            repeat(steps) { node = node.left }
            if (node.data.position == Position.DOWN) return steps
        }
    }
}

In [ ]:
Counter().basic(150)